In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # 検索ツールを準備
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # LLM を準備（ツールをバインド）
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノード
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフを定義
    graph_builder = StateGraph(State)
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードを追加
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    # ツールが必要かどうかを判定し、必要なら tools ノードへ分岐
    graph_builder.add_conditional_edges("chatbot", tools_condition)

    # ツール実行後は再度 chatbot へ戻す
    graph_builder.add_edge("tools", "chatbot")

    # エントリーポイント
    graph_builder.set_entry_point("chatbot")

    # メモリ付きでコンパイル
    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # 実行
    events = graph.stream(
        {"messages": [("user", user_input)]},f
        {"configurable": {"thread_id": "1"}},
        stream_mode="values"
    )
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)


/var/folders/md/p3ppkdgj6n30yjnbbk09tgpm0000gp/T/ipykernel_1307/3045105274.py:20: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


台湾の歴史を教えて
台湾の歴史は非常に多様で複雑です。以下に、主な歴史的な出来事や時代を簡単にまとめます。

### 先史時代
- 台湾には約5,000年前から先住民が住んでいました。先住民はオーストロネシア系の民族で、多様な文化と言語を持っています。

### 17世紀
- 1624年：オランダが台湾に進出し、南部に植民地を設立。
- 1626年：スペインが北部に進出するが、オランダの攻撃で撤退。
- 1661年：鄭成功（コスギョウ）がオランダ人を追放し、台湾を統治。

### 清朝時代 (1683年 - 1895年)
- 1683年：清朝が台湾を征服し、正式に清の領土となる。
- この時期、漢民族の移民が増加し、農業や商業が発展。

### 日本統治時代 (1895年 - 1945年)
- 1895年：下関条約により、台湾が日本に割譲される。
- 台湾は日本の植民地となり、インフラ整備や教育制度の整備が進む。

### 戦後 (1945年 - 現在)
- 1945年：日本の敗戦により、台湾は再び中国（国民政府）の支配下に。
- 1949年：中国内戦が終結し、中華人民共和国が成立。国民党政府は台湾に撤退。
- 台湾は国民政府のもとで一党独裁体制が続くが、1980年代から民主化が進展。

### 現代
- 1996年：台湾で初の直接民主選挙が行われ、民主主義が確立。
- 現在、台湾は経済的に発展した地域であり、国際的な政治的地位については議論が続いています。

このように、台湾の歴史は多くの文化や民族、そして国際的な影響を受けた非常に豊かなものです。
ありがとうございました!
